In [1]:
# Performs a simulation to come up with the number of games required to achieve each rank in a season of MTG Arena
# Using data from the Dec 18 State of the Beta

import random
import pandas as pd
MAX_GAMES = int((31 * 24 * 60) / 5) # A high end that is physically impossible for any one person to hit
WINRATE_INCREMENT = 0.01
SIMULATIONS = 1000

In [2]:
# Rank	Steps Gained with Win	Steps Lost with Loss	Steps needed to Advance a Tier
# Bronze	2	0	4
# Silver	2	1	5
# Gold	    1	1	6
# Platinum	1	1	7
# Diamond	1	1	7

# never get pushed to a lower rank

ranks = [
    {"title": "Bronze", "PerWin": 2, "PerLoss": 0, "PerTier": 4},
    {"title": "Silver", "PerWin": 2, "PerLoss": 1, "PerTier": 5},
    {"title": "Gold", "PerWin": 1, "PerLoss": 1, "PerTier": 6},
    {"title": "Platinum", "PerWin": 1, "PerLoss": 1, "PerTier": 7},
    {"title": "Diamond", "PerWin": 1, "PerLoss": 1, "PerTier": 7},
    {"title": "Mythic", "PerWin": 1, "PerLoss": 1, "PerTier": 99999999999999999999999}
]

class Player:
    def __init__(self, winrate):
        self.winrate = winrate
        self.rank = 0
        self.tier = 4
        self.steps = 0
    def getRank(self):
        return ranks[self.rank]
    def simGame(self):
        # Returns true for win false for loss
        return random.random() <= self.winrate
    def simRanked(self):
        rank = self.getRank()
        win = self.simGame()
        if win:
            self.steps += rank["PerWin"]
        else:
            self.steps -= rank["PerLoss"]

        self.steps = max(0, self.steps) 
        if self.steps < 0:
            self.downgradeTier()
        if self.steps >= rank["PerTier"]:
            self.upgradeTier()
    def upgradeTier(self):
        self.tier -= 1
        if self.tier <= 0:
            self.upgradeRank()
        self.steps = 0
    def downgradeTier(self):
        self.tier = min(4, self.tier)
        self.steps = 0
    def upgradeRank(self):
        self.rank += 1
        self.steps = 0
        self.tier = 4

In [3]:
import time
start_time = time.time()
results = []
for w in range(0,100, int(WINRATE_INCREMENT*100)):
    winrate = w/100.
    tracker = {"Silver": [], "Gold": [], "Platinum": [], "Diamond": [], "Mythic": []}
    for _ in range(SIMULATIONS):
        player = Player(winrate)
        highestrank = player.rank
        for i in range(MAX_GAMES):
            player.simRanked()
            if player.rank > highestrank:
                highestrank = player.rank
                tracker[player.getRank()["title"]].append(i+1)
    r = {k: sum(v)/float(SIMULATIONS) for k,v in tracker.items()} # Get the average games to a rank for a winrate
    r = {k: v if v > 0 else 'NA' for k,v in r.items()}
    r["Winrate"] = winrate
    results.append(r)
df_results = pd.DataFrame(results, columns=["Winrate", "Silver", "Gold", "Platinum", "Diamond", "Mythic"])
print("Took {} seconds".format(time.time() - start_time))

Took 1042.8422319889069 seconds


In [4]:
pd.options.display.max_rows = 999
df_results.set_index("Winrate")

,Silver,Gold,Platinum,Diamond,Mythic
Winrate,,,,,
0.00,NA,NA,NA,NA,NA
0.01,806.429,NA,NA,NA,NA
0.02,395.177,NA,NA,NA,NA
0.03,267.927,38.559,NA,NA,NA
0.04,201.987,566.392,NA,NA,NA
0.05,158.521,2337.07,NA,NA,NA
0.06,133.536,4263.27,NA,NA,NA
0.07,111.642,3878.98,NA,NA,NA
0.08,99.222,2952.24,NA,NA,NA
